In [73]:
import pandas as pd

stock_returns = pd.read_csv("../archive/data/stock_returns.csv", index_col=0, parse_dates=True)
z_scores_3 = pd.read_csv("../archive/data/monthly_stock_pick/z_scores_3.csv", index_col=0, parse_dates=True)
z_scores_6 = pd.read_csv("../archive/data/monthly_stock_pick/z_scores_6.csv", index_col=0, parse_dates=True)
z_scores_9 = pd.read_csv("../archive/data/monthly_stock_pick/z_scores_9.csv", index_col=0, parse_dates=True)
z_scores_12 = pd.read_csv("../archive/data/monthly_stock_pick/z_scores_12.csv", index_col=0, parse_dates=True)
z_scores_18 = pd.read_csv("../archive/data/monthly_stock_pick/z_scores_18.csv", index_col=0, parse_dates=True)
z_scores_24 = pd.read_csv("../archive/data/monthly_stock_pick/z_scores_24.csv", index_col=0, parse_dates=True)

In [74]:
accumulate_return_monthly = (stock_returns + 1).cumprod().resample("BME").last()
monthly_return = accumulate_return_monthly.pct_change().shift(-1)
monthly_return = monthly_return.dropna(how="all", axis=0)
monthly_return

,CSCO,UAL,TROW,ISRG,NVR,TPR,DVN,CE,BA,VRTX,...,WM,DOV,CRM,PGR,WAT,IEX,BWA,LRCX,BLK,PPL
2004-01-30,-0.099183,NaN,0.008244,0.007311,0.064442,0.118543,0.005667,NaN,0.042852,0.137235,...,0.033333,-0.047900,NaN,0.000121,-0.025587,0.021583,-0.029273,-0.038722,0.025508,0.017716
2004-02-27,0.017702,NaN,0.027253,-0.052484,-0.005405,0.034317,0.025959,NaN,-0.053031,-0.164153,...,0.058947,-0.010717,NaN,0.060073,0.105577,0.020657,-0.059535,-0.017599,0.032280,-0.011349
2004-03-31,-0.112855,NaN,-0.047372,-0.044785,-0.019565,0.039278,0.052451,NaN,0.039445,-0.082803,...,-0.058980,0.032499,NaN,-0.000913,0.056562,0.090149,-0.031240,-0.121417,0.016838,-0.060307
2004-04-30,0.069823,NaN,-0.060842,0.020358,0.014967,0.023709,-0.030065,NaN,0.077862,0.018519,...,0.019284,-0.022893,NaN,-0.019995,0.067903,0.009524,0.011471,0.138650,0.014470,0.007001
2004-05-31,0.059454,NaN,0.050620,0.148730,0.057783,0.036230,0.113698,NaN,0.115502,0.231818,...,0.065716,0.080595,NaN,-0.005192,0.036892,0.080189,0.056226,0.066455,0.015528,0.073995
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-07-31,0.043137,-0.030383,-0.071535,0.108005,0.065638,0.021951,-0.047842,-0.074743,-0.088458,0.000343,...,0.046284,0.012455,-0.022797,0.177844,0.029945,-0.009593,-0.035118,-0.108799,0.028876,0.073688
2024-08-30,0.053027,0.295640,0.039546,-0.002761,0.069702,0.156479,-0.116682,0.041041,-0.124899,-0.062131,...,-0.017394,0.030694,0.083985,0.006185,0.039093,0.038842,0.068610,-0.002981,0.059124,0.044976
2024-09-30,0.036989,0.371539,0.008538,0.025587,-0.067163,0.010004,-0.011247,-0.068338,-0.017956,0.023437,...,0.039740,-0.012569,0.064521,-0.042696,-0.102198,0.003976,-0.073299,-0.088937,0.033196,-0.015719
2024-10-31,0.081066,0.237286,0.127253,0.075738,0.009043,0.312540,-0.018873,-0.418830,0.041056,-0.016492,...,0.057308,0.090238,0.132546,0.107277,0.190678,0.074497,0.020517,-0.006321,0.042578,0.072789


In [76]:
from statsmodels.api import OLS

def cal_IC(month_end: pd.Timestamp, next_month_stock_returns: pd.DataFrame, z_scores: pd.DataFrame):
    z_scores_month = z_scores.loc[month_end]
    next_month_stock_returns_ = next_month_stock_returns.loc[month_end]

    data = pd.concat([z_scores_month, next_month_stock_returns_], axis=1, join="inner").dropna()
    data.columns = ["z_scores", "next_month_stock_returns"]
    
    IC = data.corr().iloc[0, 1]
    return IC
    

ICs_3 = []
ICs_6 = []
ICs_9 = []
ICs_12 = []
ICs_18 = []
ICs_24 = []
    
for month_end in monthly_return.index:
    ICs_3.append(cal_IC(month_end, monthly_return, z_scores_3))
    ICs_6.append(cal_IC(month_end, monthly_return, z_scores_6))
    ICs_9.append(cal_IC(month_end, monthly_return, z_scores_9))
    ICs_12.append(cal_IC(month_end, monthly_return, z_scores_12))
    ICs_18.append(cal_IC(month_end, monthly_return, z_scores_18))
    ICs_24.append(cal_IC(month_end, monthly_return, z_scores_24))
ICs_3 = pd.Series(ICs_3, index=monthly_return.index)
ICs_6 = pd.Series(ICs_6, index=monthly_return.index)
ICs_9 = pd.Series(ICs_9, index=monthly_return.index)
ICs_12 = pd.Series(ICs_12, index=monthly_return.index)
ICs_18 = pd.Series(ICs_18, index=monthly_return.index)
ICs_24 = pd.Series(ICs_24, index=monthly_return.index)
ICs = pd.concat([ICs_3, ICs_6, ICs_9, ICs_12, ICs_18, ICs_24], axis=1)
ICs.columns = ["IC_3", "IC_6", "IC_9", "IC_12", "IC_18", "IC_24"]

In [77]:
ICs

,IC_3,IC_6,IC_9,IC_12,IC_18,IC_24
2004-01-30,-0.055523,-0.055523,-0.055523,-0.055523,-0.055523,-0.055523
2004-02-27,-0.029516,-0.029516,-0.029516,-0.029516,-0.029516,-0.029516
2004-03-31,0.099147,0.099147,0.099147,0.099147,0.099147,0.099147
2004-04-30,-0.142997,0.006675,0.006675,0.006675,0.006675,0.006675
2004-05-31,0.241477,0.143706,0.143706,0.143706,0.143706,0.143706
...,...,...,...,...,...,...
2024-07-31,-0.139055,-0.219861,-0.200235,-0.181175,-0.210549,-0.255552
2024-08-30,0.071747,0.109809,0.103138,0.094127,0.069317,0.026056
2024-09-30,0.045141,0.035549,0.019065,0.020673,-0.035565,-0.122960
2024-10-31,-0.008507,0.034831,0.034191,0.095924,0.060014,-0.070920


In [78]:
# test the significance of IC, t-test
from scipy.stats import ttest_1samp
t_stat, p_value = ttest_1samp(ICs, 0)
t_stat, p_value

(array([0.71392597, 1.70515569, 1.54371289, 1.61068599, 1.38460441,
        0.75709199]),
 array([0.47593895, 0.08940757, 0.12392275, 0.10851001, 0.16740798,
        0.44970769]))

In [79]:
ICs.mean()

IC_3     0.005005
IC_6     0.013155
IC_9     0.012119
IC_12    0.012617
IC_18    0.011048
IC_24    0.006125
dtype: float64